# 2. Preprocessing:

We first repeat the import steps to get all the df and update the schema

In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import json

In [2]:
# Import df
END_POINT = "data.cityofnewyork.us"
NEW_APP_TOKEN = 'TOKEN'
USERNAME = 'YOUR USERNAME'
PASSWORD = 'YOUR PASSWORD'



client = Socrata(END_POINT,
                 NEW_APP_TOKEN,
                 username=USERNAME,
                 password=PASSWORD)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("7x5e-2fxh",limit=29000) #feel free to unlimit this if you'd like. 2000 is sufficient for the hackathon (if not doing ML)

# Convert to pandas DataFrame
df = pd.DataFrame.from_records(results)


Update the schema

In [3]:
# Adjusting df types
# print(list(df.dtypes))

types = {'numeric':0,'text':0,'floating_timestamp':0}
from schema import schema_dict

for column, dtype in schema_dict.items():
    if column not in df.columns:
        print('col not present',column)
        continue
    if dtype == 'number':
        types['numeric'] +=1
        df[column] = df[column].astype(float)  # or .astype(int) depending on the specifics
    elif dtype == 'text':
        types['text'] +=1

        df[column] = df[column].astype(str)
    elif dtype == 'floating_timestamp':
        types['floating_timestamp'] +=1

        df[column] = pd.to_datetime(df[column])

types


col not present bin
col not present bbl
col not present dof_benchmarking_submission_id


{'numeric': 15, 'text': 230, 'floating_timestamp': 3}

In [4]:
# Before filling
# Find the percentage of missing values for each column
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)



latitude            3.789655
longitude           3.789655
community_board     3.789655
council_district    3.789655
census_tract        3.789655
borough             3.768966
dtype: float64


In [5]:
# drop all of the na
df.dropna(inplace=True)
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


Series([], dtype: float64)


In [6]:
df.replace("Not Available", pd.NA, inplace=True)
df.replace("Not Applicable", pd.NA, regex=True, inplace=True)

# Find the percentage of missing values for each column
missing_percent = (df.isnull().sum() / len(df)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


propane_use_kbtu                   100.000000
barracks_gross_floor_area          100.000000
data_center_national_median         99.996416
data_center_pue                     99.996416
convention_center_gross_floor       99.992832
district_hot_water_use_kbtu         99.985664
municipally_supplied_potable_3      99.974911
outdoor_water_use_all_water         99.974911
wholesale_club_supercenter          99.971327
wholesale_club_supercenter_1        99.971327
fuel_oil_1_use_kbtu                 99.964159
third_party_certification_2         99.960575
third_party_certification_1         99.960575
third_party_certification           99.960575
ambulatory_surgical_center          99.956991
estimated_data_flag_fuel            99.956991
energy_power_station_gross          99.956991
wastewater_treatment_plant          99.949823
fuel_oil_5_6_use_kbtu               99.939070
automobile_dealership_gross         99.913982
adult_education_gross_floor         99.910397
mailing_center_post_office        

In [7]:
# Save the DataFrame to a CSV file
df.to_csv('./data/data_full.csv', index=False)

# Get the schema of the DataFrame
schema = {
    column: str(dtype) 
    for column, dtype in zip(df.columns, df.dtypes)
}

# Write the schema to a JSON file
with open('./data/schema.json', 'w') as schema_file:
    json.dump(schema, schema_file, indent=4)

# Focusing on multifamily

In [8]:
df_saved = pd.read_csv('./data/data_full.csv')

C:\Users\andre\AppData\Local\Temp\ipykernel_12216\2101965794.py:1: DtypeWarning: Columns (54,55,57,58,60,61,97,98,192,197,227,228,229) have mixed types. Specify dtype option on import or set low_memory=False.
  df_saved = pd.read_csv('./data/data_full.csv')


In [17]:
# Drop all columns where multifam is na. We will use 'multifamily_housing_number' as a surrogate

surrogates = ['multifamily_housing_total']
# Assuming df is your DataFrame and 'column_name' is the name of the column
for surrogate in surrogates:
    df_saved.dropna(subset=[surrogate],inplace=True)


# See the new nas
# Find the percentage of missing values for each column
missing_percent = (df_saved.isnull().sum() / len(df_saved)) * 100

# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


propane_use_kbtu                   100.000000
refrigerated_warehouse_gross       100.000000
wastewater_treatment_plant         100.000000
hospital_general_medical_2         100.000000
convention_center_gross_floor      100.000000
data_center_national_median        100.000000
distribution_center_gross          100.000000
barracks_gross_floor_area          100.000000
data_center_pue                    100.000000
district_hot_water_use_kbtu        100.000000
courthouse_gross_floor_area        100.000000
hospital_general_medical            99.994541
district_chilled_water_use          99.994541
data_center_cooling_equipment       99.994541
data_center_ups_system              99.994541
wholesale_club_supercenter          99.994541
wholesale_club_supercenter_1        99.994541
manufacturing_industrial            99.994541
hospital_general_medical_3          99.994541
hospital_general_medical_4          99.994541
hospital_general_medical_5          99.994541
hospital_general_medical_6        

In [16]:
# DROP

threshold_percentage = 2
missing_percent = (df_saved.isnull().sum() / len(df_saved)) * 100
# Get columns to drop based on threshold
columns_to_drop = missing_percent[missing_percent > threshold_percentage].index

print('dropping ', len(list(columns_to_drop)))

len(df_saved.columns)

# Drop these columns from the DataFrame
df_cleaned = df_saved.drop(columns=columns_to_drop)

len(df_cleaned.columns)


dropping  180


,property_id,property_name,year_ending,nyc_borough_block_and_lot,nyc_building_identification,address_1,city,postal_code,primary_property_type_self,primary_property_type,...,number_of_active_it_meters,generation_date,submission_date,latitude,longitude,community_board,council_district,census_tract,nta,borough
0,21205224.0,Astoria Blvd Property,2021-12-31,4006520042,4538607,36-16 Astoria Blvd.,Long Island City,11103,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.769272,-73.913633,401.0,22.0,6502.0,Astoria,QUEENS
1,2665352.0,Stellar - 70 West 93rd Street,2021-12-31,1-01206-0001,1031627,70 West 93rd Street,New York,10025,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.790503,-73.967920,107.0,6.0,177.0,Upper West Side,MANHATTAN
2,2665400.0,Stellar - 28-50 West 97th Street,2021-12-31,1-01832-0043,1055265,28-50 West 97th Street,New York,10025,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.792758,-73.965171,107.0,7.0,181.0,Upper West Side,MANHATTAN
3,2665415.0,Stellar - 87 Hamilton Place,2021-12-31,1-02072-0039,1061772,87 Hamilton Place,New York,10031,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.823008,-73.950670,109.0,7.0,225.0,Hamilton Heights,MANHATTAN
4,2665418.0,Stellar - 3458-68 Broadway,2021-12-31,1-02073-0001,1061777,3458-68 Broadway,New York,10031,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.823978,-73.952230,109.0,7.0,225.0,Hamilton Heights,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27895,6575086.0,35 WINTHROP REALTY CORP,2021-12-31,3050450091,3115687;3115686,35 WINTHROP STREET,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.656592,-73.958524,309.0,40.0,79601.0,Prospect Lefferts Gardens-Wingate,BROOKLYN
27896,6575119.0,782 EAST 32ND STREET LLC,2021-12-31,3075590001,3205933,782 EAST 32ND STREET,BROOKLYN,11210,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.632396,-73.945691,314.0,45.0,786.0,Flatbush,BROOKLYN
27897,6575799.0,TAG REALTY INC,2021-12-31,3012800066,3033472,969 CARROLL STREET,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.667850,-73.958034,309.0,35.0,325.0,Crown Heights South,BROOKLYN
27898,6576684.0,LINCOLN64 FLATS LLC,2021-12-31,3050280006,3114678,64 LINCOLN ROAD,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.660989,-73.959985,309.0,40.0,79801.0,Prospect Lefferts Gardens-Wingate,BROOKLYN


In [11]:
# See the new nas
# Find the percentage of missing values for each column
missing_percent = (df_cleaned.isnull().sum() / len(df_cleaned)) * 100
# Filter out columns that have missing values
columns_with_missing = missing_percent[missing_percent > 0].sort_values(ascending=False)
pd.set_option('display.max_rows', 300)
print(columns_with_missing)


metered_areas_energy             1.555932
indirect_ghg_emissions           0.955397
indirect_ghg_emissions_metric    0.955397
net_emissions_metric_tons        0.807993
nyc_building_identification      0.103729
nyc_borough_block_and_lot        0.098269
egrid_output_emissions_rate      0.054594
national_median_total_ghg        0.049135
last_modified_date_property_1    0.032756
last_modified_date_property      0.027297
national_median_source_eui       0.010919
national_median_site_eui         0.010919
national_median_reference        0.010919
multifamily_housing_number_7     0.010919
multifamily_housing_number_1     0.005459
multifamily_housing_total_1      0.005459
multifamily_housing_number_5     0.005459
multifamily_housing_number_6     0.005459
dtype: float64


In [12]:
df_cleaned.dropna(inplace=True)

In [13]:
df_cleaned

,property_id,property_name,year_ending,nyc_borough_block_and_lot,nyc_building_identification,address_1,city,postal_code,primary_property_type_self,primary_property_type,...,number_of_active_it_meters,generation_date,submission_date,latitude,longitude,community_board,council_district,census_tract,nta,borough
0,21205224.0,Astoria Blvd Property,2021-12-31,4006520042,4538607,36-16 Astoria Blvd.,Long Island City,11103,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.769272,-73.913633,401.0,22.0,6502.0,Astoria,QUEENS
1,2665352.0,Stellar - 70 West 93rd Street,2021-12-31,1-01206-0001,1031627,70 West 93rd Street,New York,10025,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.790503,-73.967920,107.0,6.0,177.0,Upper West Side,MANHATTAN
2,2665400.0,Stellar - 28-50 West 97th Street,2021-12-31,1-01832-0043,1055265,28-50 West 97th Street,New York,10025,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.792758,-73.965171,107.0,7.0,181.0,Upper West Side,MANHATTAN
3,2665415.0,Stellar - 87 Hamilton Place,2021-12-31,1-02072-0039,1061772,87 Hamilton Place,New York,10031,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.823008,-73.950670,109.0,7.0,225.0,Hamilton Heights,MANHATTAN
4,2665418.0,Stellar - 3458-68 Broadway,2021-12-31,1-02073-0001,1061777,3458-68 Broadway,New York,10031,Multifamily Housing,Multifamily Housing,...,0.0,2022-09-26,2022-09-26,40.823978,-73.952230,109.0,7.0,225.0,Hamilton Heights,MANHATTAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27895,6575086.0,35 WINTHROP REALTY CORP,2021-12-31,3050450091,3115687;3115686,35 WINTHROP STREET,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.656592,-73.958524,309.0,40.0,79601.0,Prospect Lefferts Gardens-Wingate,BROOKLYN
27896,6575119.0,782 EAST 32ND STREET LLC,2021-12-31,3075590001,3205933,782 EAST 32ND STREET,BROOKLYN,11210,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.632396,-73.945691,314.0,45.0,786.0,Flatbush,BROOKLYN
27897,6575799.0,TAG REALTY INC,2021-12-31,3012800066,3033472,969 CARROLL STREET,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.667850,-73.958034,309.0,35.0,325.0,Crown Heights South,BROOKLYN
27898,6576684.0,LINCOLN64 FLATS LLC,2021-12-31,3050280006,3114678,64 LINCOLN ROAD,BROOKLYN,11225,Multifamily Housing,Multifamily Housing,...,0.0,2022-03-29,2022-03-29,40.660989,-73.959985,309.0,40.0,79801.0,Prospect Lefferts Gardens-Wingate,BROOKLYN


# Save the data
We have dealt with initial features and data by looking at the N/As and equivalent values in the data.
We have filtered to only multi-family homes as well. There is bias from doing this which is unavoidable as the only way to truly know the right action is to look at each feature.

From here, we will save the data and perform EDA to consider our next steps.


In [14]:
# Save the DataFrame to a CSV file
df_cleaned.to_csv('./data/data_cleaned.csv', index=False)

# Get the schema of the DataFrame
schema = {
    column: str(dtype) 
    for column, dtype in zip(df_cleaned.columns, df_cleaned.dtypes)
}

# Write the schema to a JSON file
with open('./data/schema_cleaned.json', 'w') as schema_file:
    json.dump(schema, schema_file, indent=4)

Demo to read the data       

In [15]:
import pandas as pd
import json

# Load the schema from the JSON file
with open('./data/schema_cleaned.json') as schema_file:
    schema = json.load(schema_file)

# Convert schema to the format required by pandas
# Note: In this case, we're assuming all the dtypes are compatible with pandas dtypes.
# If there are any discrepancies, you may need to manually adjust the dtypes.
pandas_schema = {key: value for key, value in schema.items()}

# Now read the CSV file using the schema
df_new = pd.read_csv('./data/data_cleaned.csv', dtype=pandas_schema)

# Now df has the schema as specified in the schema.json file
